## Example of Testing EfficientNet Model


In [ ]:
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import cv2
from tqdm import tqdm
from datetime import datetime

In [ ]:
image_size = 260
batch_size = 32
MODEL_PATH = 'PUT_MODEL_PATH'

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")
device = get_device()

In [ ]:
class DatasetTransformer_prediction(torch.utils.data.Dataset):
    def __init__(self, df, augmented=False, transform=None):
        
        self.transform = A.Compose([
            A.Resize(image_size, image_size),
            ToTensorV2()
        ])
        self.df = df
        target_cols = df_test.iloc[:, 1:12].columns.tolist()
        self.labels = self.df[target_cols].values
        self.file_names = df['StudyInstanceUID'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_name = self.file_names[idx]
        image_path = f'../input/ranzcr-clip-catheter-line-classification/test/{image_name}.jpg'
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        image_t = self.transform(image=image)['image']
        data = { 
            "image": image_t.float(), 
        }
        
        return data

In [ ]:
class EfficientNet(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        
        model = timm.create_model(model_name, pretrained=pretrained)
            
        n_features = model.classifier.in_features
        
        self.model = nn.Sequential(*list(model.children())[:-1])
        self.drop_out = nn.Dropout(p=0.5)
        self.fc = nn.Linear(n_features, 11)
        
    def forward(self, x):
        x = self.model(x)
        x = self.drop_out(x)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
def get_prediction(model, test_dataset_loader, device):
    with torch.no_grad():
        model.eval()
        probs = []
        size = len(test_dataset_loader)
        i = 0
        for data in test_dataset_loader:
            print(f"{int(i*100/size)}%", end="\r")
            image_input = data["image"].to(device)
            output = model(image_input).to('cpu').numpy()
            probs.append(output)
            i += 1
        probs = np.concatenate(probs)
    return probs

In [ ]:
model = EfficientNet(model_name="efficientnet_b2")
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)
model.eval()

In [ ]:
df_test = pd.read_csv('./PATH_TO_DIR/sample_submission.csv')
test_dataset = DatasetTransformer_prediction(df_test)
test_dataset_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size)

In [ ]:
target_cols = df_test.iloc[:, 1:12].columns.tolist()
df_test[target_cols] = predictions
df_test[['StudyInstanceUID'] + target_cols].to_csv('submission.csv', index=False)
df_test.head(20)